In [ ]:
import tensorflow as tf
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/', nb_path)
sys.path.insert(0,nb_path)

Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


FileExistsError: ignored

In [ ]:
!ls '/content/mnt/My Drive/datasets/horse-or-human'

In [ ]:
# training Datasets
train_horse_dir = os.path.join('/content/mnt/My Drive/datasets/horse-or-human/horses')
train_human_dir = os.path.join('/content/mnt/My Drive/datasets/horse-or-human/humans')

In [ ]:
# Validation Datasets
validation_horse_dir = os.path.join('/content/mnt/My Drive/datasets/validation-horse-or-human/horses')
validation_human_dir = os.path.join('/content/mnt/My Drive/datasets/validation-horse-or-human/humans')

In [ ]:
print(train_horse_dir)
print(train_human_dir)

NameError: ignored

In [ ]:
# Now its time to see the list inside the directory...

train_horse_name = os.listdir(train_horse_dir)
train_human_name = os.listdir(train_human_dir)

# Now we will see what are the first pictures of training horses and humans...

print(train_horse_name[:10])
print(train_human_name[:10])

# shape of the both of Horses and Humans images...

# print('The shape of Horses image is: ' + str(train_horse_name))
# print('The shape of Humans image is: ' + str(train_human_name))

# Now we will see what are the first pictures of validation horses and humans...

validation_horse_name = os.listdir(validation_horse_dir)
validation_horse_name = os.listdir(validation_human_dir)

print(validation_horse_name[:10])
print(validation_horse_name[:10])

NameError: ignored

In [ ]:
# See the total length of the directory of both Horses and Humans...

print('Thie total length of training Horses Images are: ' , len(os.listdir(train_horse_dir)))
print('Thie total length of training Humans Images are: ' , len(os.listdir(train_human_dir)))

print('Thie total length of validation Horses Images are: ' , len(os.listdir(validation_horse_dir)))
print('Thie total length of validation Humans Images are: ' , len(os.listdir(validation_human_dir)))

NameError: ignored

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as npimg

# we will configuration our image output in 4x4 matrixes
nrow = 4
ncol = 4

# iteriate start from 0

pic_index = 0

In [ ]:
# assign matplotlib as 4x4 figure in order to see the output...

fig = plt.gcf()
fig.set_size_inches(ncol * 8, nrow * 8)

pic_index += 8

print(pic_index - 8)

next_horse_pixel = [os.path.join(train_horse_dir, fname) for fname in train_horse_name[pic_index - 8: pic_index]]

next_human_pixel = [os.path.join(train_human_dir, fname) for fname in train_human_name[pic_index - 8: pic_index]]

for i, img_path in enumerate(next_horse_pixel+next_human_pixel):
  # Set up the subplots and start to show the images from one (1)...

  sp = plt.subplot(nrow, ncol, i + 1)
  # Don't show any of the gulidlines...
  sp.axis('off')

  img = npimg.imread(img_path)
  plt.imshow(img)

plt.show()


0


NameError: ignored

<Figure size 2304x2304 with 0 Axes>

# Let's train our model

In [ ]:
model = tf.keras.Sequential([
   tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)),
   tf.keras.layers.MaxPooling2D(2,2),
   tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
   tf.keras.layers.MaxPooling2D(2,2),     
   tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
   tf.keras.layers.MaxPooling2D(2,2),  
   tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
   tf.keras.layers.MaxPooling2D(2,2), 
   tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
   tf.keras.layers.MaxPooling2D(2,2),  
   tf.keras.layers.Flatten(),
   tf.keras.layers.Dense(512, activation='relu'),
   tf.keras.layers.Dense(1, activation='softmax')                     
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 33, 33, 128)      

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All the images must be rescale in the [0, 1] range so that they can be easily be processable 

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    '/content/mnt/My Drive/datasets/horse-or-human',
    target_size = (300, 300), # All images will be convert it into 150x150...
    batch_size = 128, 
    # Since, we are using binary_crossentropy then we will use 'binary class mode...
    class_mode='binary'
)

validation_datagen = ImageDataGenerator(rescale=1/255)

validation_generator = validation_datagen.flow_from_directory(
    '/content/mnt/My Drive/datasets/validation-horse-or-human',
    target_size = (300, 300), # All images will be convert it into 150x150...
    batch_size = 128, 
    # Since, we are using binary_crossentropy then we will use 'binary class mode...
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 8,
    epochs = 15,
    verbose = 1,
    validation_data = validation_generator,
    validation_steps = 8
) 

Epoch 1/15
8/8 [==============================] - 10s 1s/step - loss: 7.4465 - acc: 0.5117 - val_loss: 7.6246 - val_acc: 0.5000
Epoch 2/15
8/8 [==============================] - 11s 1s/step - loss: 7.4459 - acc: 0.5117 - val_loss: 7.6246 - val_acc: 0.5000
Epoch 3/15
8/8 [==============================] - 10s 1s/step - loss: 7.3108 - acc: 0.5206 - val_loss: 7.6246 - val_acc: 0.5000
Epoch 4/15
8/8 [==============================] - 11s 1s/step - loss: 7.3787 - acc: 0.5161 - val_loss: 7.6246 - val_acc: 0.5000
Epoch 5/15
8/8 [==============================] - 10s 1s/step - loss: 7.3447 - acc: 0.5184 - val_loss: 7.6246 - val_acc: 0.5000
Epoch 6/15
8/8 [==============================] - 10s 1s/step - loss: 7.4161 - acc: 0.5137 - val_loss: 7.6246 - val_acc: 0.5000
Epoch 7/15
8/8 [==============================] - 10s 1s/step - loss: 7.4296 - acc: 0.5128 - val_loss: 7.6246 - val_acc: 0.5000
Epoch 8/15
8/8 [==============================] - 11s 1s/step - loss: 7.4974 - acc: 0.5083 - val_loss: 7

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image

upload = files.upload()

for fn in upload.keys():

  # Predicting The images...
  path = '/content/' + fn

  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  print(x)
  x = np.expand_dims(x, axis=0)

  images = np.hstack([x])
  print(x)
  classes = model.predict(x, batch_size=20)
  print(classes[0])
  if classes[0] < 0.5:
    print('It is horse')
  else:
    print('It is Human')